In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-community langchain
!pip install langgraph
!pip install -U langchain langchain-openai

In [13]:
import os
os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'LANGCHAIN_API_KEY'
os.environ["LANGCHAIN_PROJECT"] = "Langgraph_1"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [26]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import interrupt

In [36]:
class EmailState(TypedDict):
  prompt: Annotated[str, "User-provided prompt for the draft"]
  draft: Annotated[str, "LLM Generated draft email"]
  approved: Annotated[bool, "Whether the email is approved by the user"]
  final_email: Annotated[str, "Final email to send"]

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

def draft_email(state: EmailState) -> EmailState:
  user_prompt = state['prompt']
  draft = llm.invoke(user_prompt).content
  return {"draft": draft, "approved": False}

def check_for_approval(state: EmailState)-> str:
  while True:
        print("\n--- AI Generated Draft ---")
        print(state["draft"])
        print("---------------------------")
        decision = input("Do you approve this draft? (yes/no): ").strip().lower()

        if decision == "yes":
            return {"approved": True}
        else:
            revision = input("Enter revision instructions: ")
            prompt = f"""
You are refining an email draft based on human feedback.

Original draft:
{state['draft']}

Human feedback:
{revision}

Please rewrite the email draft accordingly, keeping the professional email format.
"""
            revised_draft = llm.invoke(prompt).content
            state["draft"] = revised_draft  # update draft

def finalize_email(state: EmailState):
  if state["approved"]:
    return {"final_email": state["draft"]}
  else:
    return {"final_email": "Email needs revision"}

In [44]:
# 5. Build the graph
builder = StateGraph(EmailState)
builder.add_node("generate_draft", draft_email)
builder.add_node("human_approval", check_for_approval)
builder.add_node("finalize_email", finalize_email)

builder.set_entry_point("generate_draft")
builder.add_edge("generate_draft", "human_approval")
builder.add_edge("human_approval", "finalize_email")
builder.add_edge("finalize_email", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [46]:
# Ask user if they want to resume or start fresh
choice = input("Do you want to resume previous session? (yes/no): ").strip().lower()

if choice == "no":
    # start new flow
    user_prompt = input("Enter your email request: ")
    initial_state = {
        "prompt": user_prompt,
        "draft": "",
        "approved": False,
        "final_email": ""
    }
    result = graph.invoke(initial_state, config={"configurable": {"thread_id": "thread_1"}})
else:
    # resume previous flow
    result = graph.invoke(None, config={"configurable": {"thread_id": "thread_1"}})

print("\n✅ Final Email Output:")
print(result["final_email"])

Do you want to resume previous session? (yes/no): yes

--- AI Generated Draft ---
Subject: Please Sign the Statement of Work

Dear Peter,

I hope this email finds you well. I am writing to kindly request your signature on the Statement of Work (SoW) for our upcoming project. This document outlines the scope of work, deliverables, timelines, and responsibilities for both parties involved.

Your signature is crucial in finalizing this agreement and ensuring that we are all on the same page regarding the project details. Please take a moment to review the SoW and let me know if you have any questions or concerns before signing.

Once you have signed the document, please scan and send it back to me at your earliest convenience. If you prefer to sign it in person, we can arrange a time to meet and finalize the details.

Thank you for your prompt attention to this matter. I look forward to working together on this project and achieving our shared goals.

Best regards,

[Your Name] 
[Your Tit

In [47]:
graph.get_state({
    "configurable": {"thread_id": 'thread_1'}
    })

StateSnapshot(values={'prompt': 'write email to Peter to sign the SoW', 'draft': 'Subject: Please Sign the Statement of Work\n\nDear Peter,\n\nI hope this email finds you well. I am writing to kindly request your signature on the Statement of Work (SoW) for our upcoming project. This document outlines the scope of work, deliverables, timelines, and responsibilities for both parties involved.\n\nYour signature is crucial in finalizing this agreement and ensuring that we are all on the same page regarding the project details. Please take a moment to review the SoW and let me know if you have any questions or concerns before signing.\n\nOnce you have signed the document, please scan and send it back to me at your earliest convenience. If you prefer to sign it in person, we can arrange a time to meet and finalize the details.\n\nThank you for your prompt attention to this matter. I look forward to working together on this project and achieving our shared goals.\n\nBest regards,\n\n[Your Na

In [48]:
list(graph.get_state_history({
    "configurable": {"thread_id": 'thread_1'}
    }))

[StateSnapshot(values={'prompt': 'write email to Peter to sign the SoW', 'draft': 'Subject: Please Sign the Statement of Work\n\nDear Peter,\n\nI hope this email finds you well. I am writing to kindly request your signature on the Statement of Work (SoW) for our upcoming project. This document outlines the scope of work, deliverables, timelines, and responsibilities for both parties involved.\n\nYour signature is crucial in finalizing this agreement and ensuring that we are all on the same page regarding the project details. Please take a moment to review the SoW and let me know if you have any questions or concerns before signing.\n\nOnce you have signed the document, please scan and send it back to me at your earliest convenience. If you prefer to sign it in person, we can arrange a time to meet and finalize the details.\n\nThank you for your prompt attention to this matter. I look forward to working together on this project and achieving our shared goals.\n\nBest regards,\n\n[Your N